<a href="https://colab.research.google.com/github/SKCA-Smart-Maintenance/Motor_Insulation_Analyzer/blob/main/Motor_Insulation_Analyzer_Ver_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Code

In [179]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [180]:
from google.colab import output
output.enable_custom_widget_manager()

In [181]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

In [182]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
import pandas as pd
import numpy as np
import natsort
import os
from datetime import datetime
from IPython.display import clear_output, display
from ipywidgets import Dropdown, Output, Button, Layout, ButtonStyle,GridspecLayout

In [183]:
class Analyzer:

  def __init__(self):
    self.path = "/content/drive/Shareddrives/SKCA 16 PE TPM TEAM/20.PI_TEST/Data"
    self.production_line_folder = natsort.natsorted(os.listdir(self.path))
    self.gc = gspread.authorize(creds)
    self.worksheet = self.gc.open('PI-Test Result Record').sheet1
    self.setup_gui()

  def setup_gui(self):
    self.dropdown1 = Dropdown(options=self.production_line_folder, value=None, description='Production Line:', disabled=False, style={'description_width': 'initial'}, layout=Layout(width='auto', margin = '0 5px'))
    self.dropdown1.observe(self.update_machine_list)

    self.dropdown2 = Dropdown(options=[""], value=None, description='Machine:', disabled=True, style={'description_width': 'initial'}, layout=Layout(width='auto', margin = '0 5px'))
    self.dropdown2.observe(self.update_motor_type_list)

    self.dropdown3 = Dropdown(options=[""], value=None, description='Motor Type:', disabled=True, style={'description_width': 'initial'}, layout=Layout(width='auto', margin = '0 5px'))
    self.dropdown3.observe(self.update_motor_list)

    self.dropdown4 = Dropdown(options=[""], value=None, description='Motor:', disabled=True, style={'description_width': 'initial'}, layout=Layout(width='auto', margin = '0 5px'))
    self.dropdown4.observe(self.update_file_list)

    self.dropdown5 = Dropdown(options=[""], value=None, description='File:', disabled=True, style={'description_width': 'initial'}, layout=Layout(width='auto', margin = '0 5px'))
    self.button1 = Button(description='Analyze', button_style='primary', layout=Layout(width='auto', margin = '0 5px'))
    self.button1.on_click(self.analyze_button_eventhandler)

    #self.button2 = Button(description='Generate Report', button_style='info', layout=Layout(width='auto', margin = '0 5px'))

    self.dropdown11 = Dropdown(options=self.production_line_folder, value=None, description='Production Line:', disabled=False, style={'description_width': 'initial'}, layout=Layout(width='auto', margin = '5px 5px'))
    self.dropdown11.observe(self.update_compare_machine_list)

    self.dropdown12 = Dropdown(options=[""], value=None, description='Machine:', disabled=True, style={'description_width': 'initial'}, layout=Layout(width='auto', margin = '5px 5px'))
    self.dropdown12.observe(self.update_compare_motor_type_list)

    self.dropdown13 = Dropdown(options=[""], value=None, description='Motor Type:', disabled=True, style={'description_width': 'initial'}, layout=Layout(width='auto', margin = '5px 5px'))
    self.dropdown13.observe(self.update_compare_motor_list)

    self.dropdown14 = Dropdown(options=[""], value=None, description='Motor:', disabled=True, style={'description_width': 'initial'}, layout=Layout(width='auto', margin = '5px 5px'))
    self.dropdown14.observe(self.update_compare_file_list)

    self.dropdown15 = Dropdown(options=[""], value=None, description='File:', disabled=True, style={'description_width': 'initial'}, layout=Layout(width='auto', margin = '5px 5px'))
    self.button11 = Button(description='Compare', button_style='warning', layout=Layout(width='auto', margin = '5px 5px'))
    self.button11.on_click(self.compare_button_eventhandler)

    self.button12 = Button(description='Save', button_style='success', layout=Layout(width='auto', margin='5px 5px'))
    self.button12.on_click(self.save_button_enventhandler)

    self.analyze_title = widgets.ToggleButton(value=False, description='Analyze', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))

    self.result_title = widgets.ToggleButton(value=False, description='Result', disabled=True, button_style='', layout=Layout(width='auto', margin = '10px 5px'))
    self.result = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '10px 5px'))

    self.overwrite_title = widgets.ToggleButton(value=False, description='Overwrite', disabled=True, button_style='', layout=Layout(width='auto', margin = '10px 5px'))
    self.overwrite = Dropdown(options=["", "Normal", "Warning", "Danger", "Reconfirm"], value=None, description='', disabled=False, style={'description_width': 'initial'}, layout=Layout(width='auto', margin = '10px 5px'))

    self.phase_r_title = widgets.ToggleButton(value=False, description='PHASE R', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))

    self.max_title_r = widgets.ToggleButton(value=False, description='MAX', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))
    self.mean_title_r = widgets.ToggleButton(value=False, description='MEAN', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))
    self.last_title_r = widgets.ToggleButton(value=False, description='LAST', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))
    self.dar_title_r = widgets.ToggleButton(value=False, description='DAR', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))
    self.ir_title_r = widgets.ToggleButton(value=False, description='IR', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))
    self.pi_title_r = widgets.ToggleButton(value=False, description='PI', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))
    self.r1_title_r = widgets.ToggleButton(value=False, description='R1', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))
    self.r5_title_r = widgets.ToggleButton(value=False, description='R5', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))
    self.r10_title_r = widgets.ToggleButton(value=False, description='R10', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))

    self.max_r = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.mean_r = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.last_r = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.dar_r = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.ir_r = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.pi_r = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.r1_r = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.r5_r = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.r10_r = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))

    self.phase_s_title = widgets.ToggleButton(value=False, description='PHASE S', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))

    self.max_title_s = widgets.ToggleButton(value=False, description='MAX', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))
    self.mean_title_s = widgets.ToggleButton(value=False, description='MEAN', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))
    self.last_title_s = widgets.ToggleButton(value=False, description='LAST', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))
    self.dar_title_s = widgets.ToggleButton(value=False, description='DAR', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))
    self.ir_title_s = widgets.ToggleButton(value=False, description='IR', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))
    self.pi_title_s = widgets.ToggleButton(value=False, description='PI', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))
    self.r1_title_s = widgets.ToggleButton(value=False, description='R1', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))
    self.r5_title_s = widgets.ToggleButton(value=False, description='R5', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))
    self.r10_title_s = widgets.ToggleButton(value=False, description='R10', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))

    self.max_s = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.mean_s = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.last_s = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.dar_s = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.ir_s = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.pi_s = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.r1_s = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.r5_s = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.r10_s = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))

    self.phase_t_title = widgets.ToggleButton(value=False, description='PHASE T', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))

    self.max_title_t = widgets.ToggleButton(value=False, description='MAX', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))
    self.mean_title_t = widgets.ToggleButton(value=False, description='MEAN', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))
    self.last_title_t = widgets.ToggleButton(value=False, description='LAST', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))
    self.dar_title_t = widgets.ToggleButton(value=False, description='DAR', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))
    self.ir_title_t = widgets.ToggleButton(value=False, description='IR', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))
    self.pi_title_t = widgets.ToggleButton(value=False, description='PI', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))
    self.r1_title_t = widgets.ToggleButton(value=False, description='R1', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))
    self.r5_title_t = widgets.ToggleButton(value=False, description='R5', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))
    self.r10_title_t = widgets.ToggleButton(value=False, description='R10', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))

    self.max_t = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.mean_t = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.last_t = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.dar_t = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.ir_t = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.pi_t = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.r1_t = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.r5_t = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.r10_t = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))

    self.fig_analyze = make_subplots(rows=4, cols=1, subplot_titles=("All Phase (MΩ)", "Phase R (MΩ)", "Phase S (MΩ)", "Phase T (MΩ)"), vertical_spacing=0.05)

    self.fig_analyze.add_trace(go.Scatter(x=[], y=[], mode='lines+markers', name='Line 1'), row=1, col=1)
    self.fig_analyze.add_trace(go.Scatter(x=[], y=[], mode='lines+markers', name='Line 2'), row=2, col=1)
    self.fig_analyze.add_trace(go.Scatter(x=[], y=[], mode='lines+markers', name='Line 3'), row=3, col=1)
    self.fig_analyze.add_trace(go.Scatter(x=[], y=[], mode='lines+markers', name='Line 4'),row=4, col=1)
    self.fig_analyze.update_layout(height=900, width=1000, showlegend=False, margin=dict(l=10, r=10, t=30, b=10))
    self.fig_analyze.update_xaxes(range=[-1, 305], showgrid=True, zeroline=True, showline=True, row=1, col=1)
    self.fig_analyze.update_yaxes(range=[-1, 305], showgrid=True, zeroline=True, showline=True, row=1, col=1)
    self.fig_analyze.update_xaxes(range=[-1, 305], showgrid=True, zeroline=True, showline=True, row=2, col=1)
    self.fig_analyze.update_yaxes(range=[-1, 305], showgrid=True, zeroline=True, showline=True, row=2, col=1)
    self.fig_analyze.update_xaxes(range=[-1, 305], showgrid=True, zeroline=True, showline=True, row=3, col=1)
    self.fig_analyze.update_yaxes(range=[-1, 305], showgrid=True, zeroline=True, showline=True, row=3, col=1)
    self.fig_analyze.update_xaxes(range=[-1, 305], showgrid=True, zeroline=True, showline=True, row=4, col=1)
    self.fig_analyze.update_yaxes(range=[-1, 305], showgrid=True, zeroline=True, showline=True, row=4, col=1)
    self.plot_analyze = Output()
    with self.plot_analyze:
        self.fig_analyze.show()

    self.compare_title = widgets.ToggleButton(value=False, description='Compare', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))

    self.phase_r_title_2 = widgets.ToggleButton(value=False, description='PHASE R', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))
    self.ir_title_r_2 = widgets.ToggleButton(value=False, description='IR', disabled=True, button_style='info', layout=Layout(width='auto', margin = '0 5px'))
    self.pi_title_r_2 = widgets.ToggleButton(value=False, description='PI', disabled=True, button_style='info', layout=Layout(width='auto', margin = '0 5px'))
    self.r1_title_r_2 = widgets.ToggleButton(value=False, description='R1', disabled=True, button_style='info', layout=Layout(width='auto', margin = '0 5px'))
    self.r5_title_r_2 = widgets.ToggleButton(value=False, description='R5', disabled=True, button_style='info', layout=Layout(width='auto', margin = '0 5px'))
    self.r10_title_r_2 = widgets.ToggleButton(value=False, description='R10', disabled=True, button_style='info', layout=Layout(width='auto', margin = '0 5px'))

    self.ir_title_r_3 = widgets.ToggleButton(value=False, description='IR', disabled=True, button_style='warning', layout=Layout(width='auto', margin = '0 5px'))
    self.pi_title_r_3 = widgets.ToggleButton(value=False, description='PI', disabled=True, button_style='warning', layout=Layout(width='auto', margin = '0 5px'))
    self.r1_title_r_3 = widgets.ToggleButton(value=False, description='R1', disabled=True, button_style='warning', layout=Layout(width='auto', margin = '0 5px'))
    self.r5_title_r_3 = widgets.ToggleButton(value=False, description='R5', disabled=True, button_style='warning', layout=Layout(width='auto', margin = '0 5px'))
    self.r10_title_r_3 = widgets.ToggleButton(value=False, description='R10', disabled=True, button_style='warning', layout=Layout(width='auto', margin = '0 5px'))

    self.ir_r_2 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.pi_r_2 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.r1_r_2 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.r5_r_2 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.r10_r_2 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))

    self.ir_r_3 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.pi_r_3 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.r1_r_3 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.r5_r_3 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.r10_r_3 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))

    self.phase_s_title_2 = widgets.ToggleButton(value=False, description='PHASE S', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))
    self.ir_title_s_2 = widgets.ToggleButton(value=False, description='IR', disabled=True, button_style='info', layout=Layout(width='auto', margin = '0 5px'))
    self.pi_title_s_2 = widgets.ToggleButton(value=False, description='PI', disabled=True, button_style='info', layout=Layout(width='auto', margin = '0 5px'))
    self.r1_title_s_2 = widgets.ToggleButton(value=False, description='R1', disabled=True, button_style='info', layout=Layout(width='auto', margin = '0 5px'))
    self.r5_title_s_2 = widgets.ToggleButton(value=False, description='R5', disabled=True, button_style='info', layout=Layout(width='auto', margin = '0 5px'))
    self.r10_title_s_2 = widgets.ToggleButton(value=False, description='R10', disabled=True, button_style='info', layout=Layout(width='auto', margin = '0 5px'))

    self.ir_title_s_3 = widgets.ToggleButton(value=False, description='IR', disabled=True, button_style='warning', layout=Layout(width='auto', margin = '0 5px'))
    self.pi_title_s_3 = widgets.ToggleButton(value=False, description='PI', disabled=True, button_style='warning', layout=Layout(width='auto', margin = '0 5px'))
    self.r1_title_s_3 = widgets.ToggleButton(value=False, description='R1', disabled=True, button_style='warning', layout=Layout(width='auto', margin = '0 5px'))
    self.r5_title_s_3 = widgets.ToggleButton(value=False, description='R5', disabled=True, button_style='warning', layout=Layout(width='auto', margin = '0 5px'))
    self.r10_title_s_3 = widgets.ToggleButton(value=False, description='R10', disabled=True, button_style='warning', layout=Layout(width='auto', margin = '0 5px'))

    self.ir_s_2 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.pi_s_2 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.r1_s_2 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.r5_s_2 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.r10_s_2 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))

    self.ir_s_3 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.pi_s_3 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.r1_s_3 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.r5_s_3 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.r10_s_3 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))

    self.phase_t_title_2 = widgets.ToggleButton(value=False, description='PHASE T', disabled=True, button_style='', layout=Layout(width='auto', margin = '0 5px'))
    self.ir_title_t_2 = widgets.ToggleButton(value=False, description='IR', disabled=True, button_style='info', layout=Layout(width='auto', margin = '0 5px'))
    self.pi_title_t_2 = widgets.ToggleButton(value=False, description='PI', disabled=True, button_style='info', layout=Layout(width='auto', margin = '0 5px'))
    self.r1_title_t_2 = widgets.ToggleButton(value=False, description='R1', disabled=True, button_style='info', layout=Layout(width='auto', margin = '0 5px'))
    self.r5_title_t_2 = widgets.ToggleButton(value=False, description='R5', disabled=True, button_style='info', layout=Layout(width='auto', margin = '0 5px'))
    self.r10_title_t_2 = widgets.ToggleButton(value=False, description='R10', disabled=True, button_style='info', layout=Layout(width='auto', margin = '0 5px'))

    self.ir_title_t_3 = widgets.ToggleButton(value=False, description='IR', disabled=True, button_style='warning', layout=Layout(width='auto', margin = '0 5px'))
    self.pi_title_t_3 = widgets.ToggleButton(value=False, description='PI', disabled=True, button_style='warning', layout=Layout(width='auto', margin = '0 5px'))
    self.r1_title_t_3 = widgets.ToggleButton(value=False, description='R1', disabled=True, button_style='warning', layout=Layout(width='auto', margin = '0 5px'))
    self.r5_title_t_3 = widgets.ToggleButton(value=False, description='R5', disabled=True, button_style='warning', layout=Layout(width='auto', margin = '0 5px'))
    self.r10_title_t_3 = widgets.ToggleButton(value=False, description='R10', disabled=True, button_style='warning', layout=Layout(width='auto', margin = '0 5px'))

    self.ir_t_2 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.pi_t_2 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.r1_t_2 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.r5_t_2 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.r10_t_2 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))

    self.ir_t_3 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.pi_t_3 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.r1_t_3 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.r5_t_3 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))
    self.r10_t_3 = widgets.Text(value='RESULT', description='', disabled=True, layout=Layout(width='auto', margin = '0 5px'))

    self.fig_compare = go.Figure()
    self.fig_compare.update_layout(
        margin=dict(l=10, r=10, t=30, b=10),
        xaxis=dict(range=[-1, 305], showgrid=True, zeroline=True, showline=True),
        yaxis=dict(range=[-1, 305], showgrid=True, zeroline=True, showline=True),
        width=1000, height=900
        )
    self.plot_compare = Output()
    with self.plot_compare:
      self.fig_compare.show()

    self.grid = GridspecLayout(42, 12)
    self.grid[0,:2] = self.dropdown1
    self.grid[0,2:4] = self.dropdown2
    self.grid[0,4:6] = self.dropdown3
    self.grid[0,6:8] = self.dropdown4
    self.grid[0,8:10] = self.dropdown5
    self.grid[0,10] = self.button1
    #self.grid[0,11] = self.button2

    self.grid[1,:2] = self.dropdown11
    self.grid[1,2:4] = self.dropdown12
    self.grid[1,4:6] = self.dropdown13
    self.grid[1,6:8] = self.dropdown14
    self.grid[1,8:10] = self.dropdown15
    self.grid[1,10] = self.button11
    self.grid[1,11] = self.button12

    self.grid[2, :12] = self.analyze_title

    self.grid[3:21, :8] = self.plot_analyze

    self.grid[3, 8] = self.result_title
    self.grid[3, 9] = self.result
    self.grid[3, 10] = self.overwrite_title
    self.grid[3, 11] = self.overwrite

    self.grid[4, 8:12] = self.phase_r_title
    self.grid[5, 8] = self.max_title_r
    self.grid[5, 9] = self.max_r
    self.grid[6, 8] = self.mean_title_r
    self.grid[6, 9] = self.mean_r
    self.grid[7, 8] = self.last_title_r
    self.grid[7, 9] = self.last_r
    self.grid[8, 8] = self.dar_title_r
    self.grid[8, 9] = self.dar_r
    self.grid[5, 10] = self.ir_title_r
    self.grid[5, 11] = self.ir_r
    self.grid[6, 10] = self.pi_title_r
    self.grid[6, 11] = self.pi_r
    self.grid[7, 10] = self.r1_title_r
    self.grid[7, 11] = self.r1_r
    self.grid[8, 10] = self.r5_title_r
    self.grid[8, 11] = self.r5_r
    self.grid[9, 10] = self.r10_title_r
    self.grid[9, 11] = self.r10_r

    self.grid[10, 8:12] = self.phase_s_title
    self.grid[11, 8] = self.max_title_s
    self.grid[11, 9] = self.max_s
    self.grid[12, 8] = self.mean_title_s
    self.grid[12, 9] = self.mean_s
    self.grid[13, 8] = self.last_title_s
    self.grid[13, 9] = self.last_s
    self.grid[14, 8] = self.dar_title_s
    self.grid[14, 9] = self.dar_s
    self.grid[11, 10] = self.ir_title_s
    self.grid[11, 11] = self.ir_s
    self.grid[12, 10] = self.pi_title_s
    self.grid[12, 11] = self.pi_s
    self.grid[13, 10] = self.r1_title_s
    self.grid[13, 11] = self.r1_s
    self.grid[14, 10] = self.r5_title_s
    self.grid[14, 11] = self.r5_s
    self.grid[15, 10] = self.r10_title_s
    self.grid[15, 11] = self.r10_s

    self.grid[16, 8:12] = self.phase_t_title
    self.grid[17, 8] = self.max_title_t
    self.grid[17, 9] = self.max_t
    self.grid[18, 8] = self.mean_title_t
    self.grid[18, 9] = self.mean_t
    self.grid[19, 8] = self.last_title_t
    self.grid[19, 9] = self.last_t
    self.grid[20, 8] = self.dar_title_t
    self.grid[20, 9] = self.dar_t
    self.grid[17, 10] = self.ir_title_t
    self.grid[17, 11] = self.ir_t
    self.grid[18, 10] = self.pi_title_t
    self.grid[18, 11] = self.pi_t
    self.grid[19, 10] = self.r1_title_t
    self.grid[19, 11] = self.r1_t
    self.grid[20, 10] = self.r5_title_t
    self.grid[20, 11] = self.r5_t
    self.grid[21, 10] = self.r10_title_t
    self.grid[21, 11] = self.r10_t

    self.grid[22, :12] = self.compare_title
    self.grid[23:, :8] = self.plot_compare

    self.grid[23, 8:12] = self.phase_r_title_2
    self.grid[24, 8] = self.ir_title_r_2
    self.grid[24, 9] = self.ir_r_2
    self.grid[25, 8] = self.pi_title_r_2
    self.grid[25, 9] = self.pi_r_2
    self.grid[26, 8] = self.r1_title_r_2
    self.grid[26, 9] = self.r1_r_2
    self.grid[27, 8] = self.r5_title_r_2
    self.grid[27, 9] = self.r5_r_2
    self.grid[28, 8] = self.r10_title_r_2
    self.grid[28, 9] = self.r10_r_2
    self.grid[24, 10] = self.ir_title_r_3
    self.grid[24, 11] = self.ir_r_3
    self.grid[25, 10] = self.pi_title_r_3
    self.grid[25, 11] = self.pi_r_3
    self.grid[26, 10] = self.r1_title_r_3
    self.grid[26, 11] = self.r1_r_3
    self.grid[27, 10] = self.r5_title_r_3
    self.grid[27, 11] = self.r5_r_3
    self.grid[28, 10] = self.r10_title_r_3
    self.grid[28, 11] = self.r10_r_3

    self.grid[29, 8:12] = self.phase_s_title_2
    self.grid[30, 8] = self.ir_title_s_2
    self.grid[30, 9] = self.ir_s_2
    self.grid[31, 8] = self.pi_title_s_2
    self.grid[31, 9] = self.pi_s_2
    self.grid[32, 8] = self.r1_title_s_2
    self.grid[32, 9] = self.r1_s_2
    self.grid[33, 8] = self.r5_title_s_2
    self.grid[33, 9] = self.r5_s_2
    self.grid[34, 8] = self.r10_title_s_2
    self.grid[34, 9] = self.r10_s_2
    self.grid[30, 10] = self.ir_title_s_3
    self.grid[30, 11] = self.ir_s_3
    self.grid[31, 10] = self.pi_title_s_3
    self.grid[31, 11] = self.pi_s_3
    self.grid[32, 10] = self.r1_title_s_3
    self.grid[32, 11] = self.r1_s_3
    self.grid[33, 10] = self.r5_title_s_3
    self.grid[33, 11] = self.r5_s_3
    self.grid[34, 10] = self.r10_title_s_3
    self.grid[34, 11] = self.r10_s_3

    self.grid[35, 8:12] = self.phase_t_title_2
    self.grid[36, 8] = self.ir_title_t_2
    self.grid[36, 9] = self.ir_t_2
    self.grid[37, 8] = self.pi_title_t_2
    self.grid[37, 9] = self.pi_t_2
    self.grid[38, 8] = self.r1_title_t_2
    self.grid[38, 9] = self.r1_t_2
    self.grid[39, 8] = self.r5_title_t_2
    self.grid[39, 9] = self.r5_t_2
    self.grid[40, 8] = self.r10_title_t_2
    self.grid[40, 9] = self.r10_t_2
    self.grid[36, 10] = self.ir_title_t_3
    self.grid[36, 11] = self.ir_t_3
    self.grid[37, 10] = self.pi_title_t_3
    self.grid[37, 11] = self.pi_t_3
    self.grid[38, 10] = self.r1_title_t_3
    self.grid[38, 11] = self.r1_t_3
    self.grid[39, 10] = self.r5_title_t_3
    self.grid[39, 11] = self.r5_t_3
    self.grid[40, 10] = self.r10_title_t_3
    self.grid[40, 11] = self.r10_t_3

    self.gui = self.grid

  def update_machine_list(self, change):
    if self.dropdown1.value != None:
      self.dropdown2.options = natsort.natsorted(os.listdir(self.path + "/" + self.dropdown1.value))
      self.dropdown2.disabled, self.dropdown3.disabled, self.dropdown4.disabled, self.dropdown5.disabled = False, True, True, True
      self.dropdown2.value, self.dropdown3.value, self.dropdown4.value, self.dropdown5.value = None, None, None, None

  def update_motor_type_list(self, change):
    if self.dropdown1.value != None and self.dropdown2.value != None:
      self.dropdown3.options = natsort.natsorted(os.listdir(self.path + "/" + self.dropdown1.value + "/" + self.dropdown2.value))
      self.dropdown3.disabled, self.dropdown4.disabled, self.dropdown5.disabled = False, True, True
      self.dropdown3.value, self.dropdown4.value, self.dropdown5.value = None, None, None

  def update_motor_list(self, change):
    if self.dropdown1.value != None and self.dropdown2.value != None and self.dropdown3.value != None:
      self.dropdown4.options = natsort.natsorted(os.listdir(self.path + "/" + self.dropdown1.value + "/" + self.dropdown2.value + "/" + self.dropdown3.value))
      self.dropdown4.disabled, self.dropdown5.disabled = False, True
      self.dropdown4.value, self.dropdown5.value = None, None

  def update_file_list(self, change):
    if self.dropdown1.value != None and self.dropdown2.value != None and self.dropdown3.value != None and self.dropdown4.value != None:
      self.file_folder_path = self.path + "/" + self.dropdown1.value + "/" + self.dropdown2.value + "/" + self.dropdown3.value + "/" + self.dropdown4.value
      self.dropdown5.options = natsort.natsorted(os.listdir(self.path + "/" + self.dropdown1.value + "/" + self.dropdown2.value + "/" + self.dropdown3.value + "/" + self.dropdown4.value))
      self.dropdown5.disabled = False
      self.dropdown5.value = None

  def analyze_button_eventhandler(self, obj):
    self.overwrite.value = ""
    self.df1 = self.clean_data(self.path + "/" + self.dropdown1.value + "/" + self.dropdown2.value + "/" + self.dropdown3.value + "/" + self.dropdown4.value + "/" + self.dropdown5.value)
    self.last_modified = os.path.getmtime(self.path + "/" + self.dropdown1.value + "/" + self.dropdown2.value + "/" + self.dropdown3.value + "/" + self.dropdown4.value + "/" + self.dropdown5.value)
    self.last_modified_time = datetime.fromtimestamp(self.last_modified).strftime("%Y-%m-%d %H:%M:%S")
    self.result_value = self.get_result(self.df1)
    self.update_analyze_ui(self.df1)

  def save_button_enventhandler(self, obj):
    if self.pi_r.value == '0.00':
      values = [self.measure_datetime, self.dropdown2.value, self.dropdown4.value, self.ir_r.value,self.ir_s.value,self.ir_t.value, self.mean_r.value, self.mean_s.value, self.mean_t.value, self.result_value, self.overwrite.value, datetime.now().strftime("%Y-%m-%d %H:%M:%S")]
      self.worksheet.append_row(values)
    else:
      values = [self.measure_datetime, self.dropdown2.value, self.dropdown4.value, self.pi_r.value,self.pi_s.value,self.pi_t.value, self.mean_r.value, self.mean_s.value, self.mean_t.value, self.result_value, self.overwrite.value, datetime.now().strftime("%Y-%m-%d %H:%M:%S")]
      self.worksheet.append_row(values)

  def update_analyze_ui(self, df):
    if len(df)<=61:
      self.fig_analyze = make_subplots(rows=4, cols=1, subplot_titles=("All Phase (MΩ)", "Phase R (MΩ)", "Phase S (MΩ)", "Phase T (MΩ)"), vertical_spacing=0.05)
      self.fig_analyze.add_trace(go.Scatter(x=np.arange(0, 305, 5), y=df['R'], mode='lines+markers', name='R', line=dict(color='blue')), row=1, col=1)
      self.fig_analyze.add_trace(go.Scatter(x=np.arange(0, 305, 5), y=df['T'], mode='lines+markers', name='T', line=dict(color='orange')), row=1, col=1)
      self.fig_analyze.add_trace(go.Scatter(x=np.arange(0, 305, 5), y=df['S'], mode='lines+markers', name='S', line=dict(color='green')), row=1, col=1)
      self.fig_analyze.add_trace(go.Scatter(x=np.arange(0,305,5), y=df['R'], mode='lines+markers', name='Line 2', line=dict(color='blue')), row=2, col=1)
      self.fig_analyze.add_trace(go.Scatter(x=np.arange(0,305,5), y=df['S'], mode='lines+markers', name='Line 3', line=dict(color='green')), row=3, col=1)
      self.fig_analyze.add_trace(go.Scatter(x=np.arange(0,305,5), y=df['T'], mode='lines+markers', name='Line 4', line=dict(color='orange')),row=4, col=1)
      self.fig_analyze.update_layout(height=900, width=1000, showlegend=False, margin=dict(l=10, r=10, t=30, b=10))
      self.plot_analyze = Output()
      with self.plot_analyze:
          self.fig_analyze.show()
    else:
      self.fig_analyze = make_subplots(rows=4, cols=1, subplot_titles=("All Phase (MΩ)", "Phase R (MΩ)", "Phase S (MΩ)", "Phase T (MΩ)"), vertical_spacing=0.05)
      self.fig_analyze.add_trace(go.Scatter(x=np.arange(0, 605, 5), y=df['R'], mode='lines+markers', name='R', line=dict(color='blue')), row=1, col=1)
      self.fig_analyze.add_trace(go.Scatter(x=np.arange(0, 605, 5), y=df['T'], mode='lines+markers', name='T', line=dict(color='orange')), row=1, col=1)
      self.fig_analyze.add_trace(go.Scatter(x=np.arange(0, 605, 5), y=df['S'], mode='lines+markers', name='S', line=dict(color='green')), row=1, col=1)
      self.fig_analyze.add_trace(go.Scatter(x=np.arange(0,605,5), y=df['R'], mode='lines+markers', name='Line 2', line=dict(color='blue')), row=2, col=1)
      self.fig_analyze.add_trace(go.Scatter(x=np.arange(0,605,5), y=df['S'], mode='lines+markers', name='Line 3', line=dict(color='green')), row=3, col=1)
      self.fig_analyze.add_trace(go.Scatter(x=np.arange(0,605,5), y=df['T'], mode='lines+markers', name='Line 4', line=dict(color='orange')),row=4, col=1)
      self.fig_analyze.update_layout(height=900, width=1000, showlegend=False, margin=dict(l=10, r=10, t=30, b=10))
      self.plot_analyze = Output()
      with self.plot_analyze:
          self.fig_analyze.show()

    self.result.value = self.result_value

    self.max_r.value = f"{max(df['R']):,.2f}"
    self.mean_r.value = f"{np.mean(df['R'][12:]):,.2f}"
    self.last_r.value = f"{df['R'].iloc[-1]:,.2f}"
    self.dar_r.value = f"{(df['R'][12]/df['R'][6]):,.2f}"

    self.max_s.value = f"{max(df['S']):,.2f}"
    self.mean_s.value = f"{np.mean(df['S'][12:]):,.2f}"
    self.last_s.value = f"{df['S'].iloc[-1]:,.2f}"
    self.dar_s.value = f"{(df['S'][12]/df['S'][6]):,.2f}"

    self.max_t.value = f"{max(df['T']):,.2f}"
    self.mean_t.value = f"{np.mean(df['T'][12:]):,.2f}"
    self.last_t.value = f"{df['T'].iloc[-1]:,.2f}"
    self.dar_t.value = f"{(df['T'][12]/df['T'][6]):,.2f}"

    if len(df)<=61:
      self.ir_r.value = f"{(df['R'].iloc[-1]/df['R'][12]):,.2f}"
      self.pi_r.value = f"{0:,.2f}"
      self.r1_r.value = f"{df['R'][12]:,.2f}"
      self.r5_r.value = f"{df['R'].iloc[-1]:,.2f}"
      self.r10_r.value = f"{0:,.2f}"

      self.ir_s.value = f"{(df['S'].iloc[-1]/df['S'][12]):,.2f}"
      self.pi_s.value = f"{0:,.2f}"
      self.r1_s.value = f"{df['S'][12]:,.2f}"
      self.r5_s.value = f"{df['S'].iloc[-1]:,.2f}"
      self.r10_s.value = f"{0:,.2f}"

      self.ir_t.value = f"{(df['T'].iloc[-1]/df['T'][12]):,.2f}"
      self.pi_t.value = f"{0:,.2f}"
      self.r1_t.value = f"{df['T'][12]:,.2f}"
      self.r5_t.value = f"{df['T'].iloc[-1]:,.2f}"
      self.r10_t.value = f"{0:,.2f}"
    else:
      self.ir_r.value = f"{(df['R'][60]/df['R'][12]):,.2f}"
      self.pi_r.value = f"{(df['R'].iloc[-1]/df['R'][12]):,.2f}"
      self.r1_r.value = f"{df['R'][12]:,.2f}"
      self.r5_r.value = f"{df['R'][60]:,.2f}"
      self.r10_r.value = f"{df['R'].iloc[-1]:,.2f}"

      self.ir_s.value = f"{(df['S'][60]/df['S'][12]):,.2f}"
      self.pi_s.value = f"{(df['S'].iloc[-1]/df['S'][12]):,.2f}"
      self.r1_s.value = f"{df['S'][12]:,.2f}"
      self.r5_s.value = f"{df['S'][60]:,.2f}"
      self.r10_s.value = f"{df['S'].iloc[-1]:,.2f}"

      self.ir_t.value = f"{(df['T'][60]/df['T'][12]):,.2f}"
      self.pi_t.value = f"{(df['T'].iloc[-1]/df['T'][12]):,.2f}"
      self.r1_t.value = f"{df['T'][12]:,.2f}"
      self.r5_t.value = f"{df['T'][60]:,.2f}"
      self.r10_t.value = f"{df['T'].iloc[-1]:,.2f}"

    self.grid[3:21, :8] = self.plot_analyze

    self.gui = self.grid
    clear_output(wait=True)
    self.display_gui()

  def update_compare_machine_list(self, change):
    if self.dropdown11.value != None:
      self.dropdown12.options = natsort.natsorted(os.listdir(self.path + "/" + self.dropdown11.value))
      self.dropdown12.disabled, self.dropdown13.disabled, self.dropdown14.disabled, self.dropdown15.disabled = False, True, True, True
      self.dropdown12.value, self.dropdown13.value, self.dropdown14.value, self.dropdown15.value = None, None, None, None

  def update_compare_motor_type_list(self, change):
    if self.dropdown11.value != None and self.dropdown12.value != None:
      self.dropdown13.options = natsort.natsorted(os.listdir(self.path + "/" + self.dropdown11.value + "/" + self.dropdown12.value))
      self.dropdown13.disabled, self.dropdown14.disabled, self.dropdown15.disabled = False, True, True
      self.dropdown13.value, self.dropdown14.value, self.dropdown15.value = None, None, None

  def update_compare_motor_list(self, change):
    if self.dropdown11.value != None and self.dropdown12.value != None and self.dropdown13.value != None:
      self.dropdown14.options = natsort.natsorted(os.listdir(self.path + "/" + self.dropdown11.value + "/" + self.dropdown12.value + "/" + self.dropdown13.value))
      self.dropdown14.disabled, self.dropdown15.disabled = False, True
      self.dropdown14.value, self.dropdown15.value = None, None

  def update_compare_file_list(self, change):
    if self.dropdown11.value != None and self.dropdown12.value != None and self.dropdown13.value != None and self.dropdown14.value != None:
      self.file_folder_path = self.path + "/" + self.dropdown11.value + "/" + self.dropdown12.value + "/" + self.dropdown13.value + "/" + self.dropdown14.value
      self.dropdown15.options = natsort.natsorted(os.listdir(self.path + "/" + self.dropdown11.value + "/" + self.dropdown12.value + "/" + self.dropdown13.value + "/" + self.dropdown14.value))
      self.dropdown15.disabled = False
      self.dropdown15.value = None

  def compare_button_eventhandler(self, obj):
    self.df1 = self.clean_data(self.path + "/" + self.dropdown1.value + "/" + self.dropdown2.value + "/" + self.dropdown3.value + "/" + self.dropdown4.value + "/" + self.dropdown5.value)
    self.df2 = self.clean_data(self.path + "/" + self.dropdown11.value + "/" + self.dropdown12.value + "/" + self.dropdown13.value + "/" + self.dropdown14.value + "/" + self.dropdown15.value,)

    self.update_compare_ui(self.df1, self.df2)

  def update_compare_ui(self, df1, df2):
    if len(df1)<=61:
      self.fig_compare = make_subplots(rows=2, cols=1, subplot_titles=("Compare (MΩ)", ""), vertical_spacing=0.05)
      self.fig_compare.add_trace(go.Scatter(x=np.arange(0, 305, 5), y=df1['R'], mode='lines+markers', name='R_1', line=dict(color='blue')), row=1, col=1)
      self.fig_compare.add_trace(go.Scatter(x=np.arange(0, 305, 5), y=df1['T'], mode='lines+markers', name='T_1', line=dict(color='blue')), row=1, col=1)
      self.fig_compare.add_trace(go.Scatter(x=np.arange(0, 305, 5), y=df1['S'], mode='lines+markers', name='S_1', line=dict(color='blue')), row=1, col=1)
      self.fig_compare.add_trace(go.Scatter(x=np.arange(0, 305, 5), y=df2['R'], mode='lines+markers', name='R_2', line=dict(color='orange')), row=1, col=1)
      self.fig_compare.add_trace(go.Scatter(x=np.arange(0, 305, 5), y=df2['S'], mode='lines+markers', name='S_2', line=dict(color='orange')), row=1, col=1)
      self.fig_compare.add_trace(go.Scatter(x=np.arange(0, 305, 5), y=df2['T'], mode='lines+markers', name='T_2', line=dict(color='orange')),row=1, col=1)
      self.fig_compare.update_layout(height=900, width=1000, showlegend=True, margin=dict(l=10, r=10, t=30, b=10), hovermode="x unified")
      self.plot_compare = Output()
      with self.plot_compare:
          self.fig_compare.show()
    else:
      self.fig_compare = make_subplots(rows=2, cols=1, subplot_titles=("Compare (MΩ)", ""), vertical_spacing=0.05)
      self.fig_compare.add_trace(go.Scatter(x=np.arange(0, 605, 5), y=df1['R'], mode='lines+markers', name='R_1', line=dict(color='blue')), row=1, col=1)
      self.fig_compare.add_trace(go.Scatter(x=np.arange(0, 605, 5), y=df1['T'], mode='lines+markers', name='T_1', line=dict(color='blue')), row=1, col=1)
      self.fig_compare.add_trace(go.Scatter(x=np.arange(0, 605, 5), y=df1['S'], mode='lines+markers', name='S_1', line=dict(color='blue')), row=1, col=1)
      self.fig_compare.add_trace(go.Scatter(x=np.arange(0, 605, 5), y=df2['R'], mode='lines+markers', name='R_2', line=dict(color='orange')), row=1, col=1)
      self.fig_compare.add_trace(go.Scatter(x=np.arange(0, 605, 5), y=df2['S'], mode='lines+markers', name='S_2', line=dict(color='orange')), row=1, col=1)
      self.fig_compare.add_trace(go.Scatter(x=np.arange(0, 605, 5), y=df2['T'], mode='lines+markers', name='T_2', line=dict(color='orange')),row=1, col=1)
      self.fig_compare.update_layout(height=900, width=1000, showlegend=True, margin=dict(l=10, r=10, t=30, b=10), hovermode="x unified")
      self.plot_compare = Output()
      with self.plot_compare:
          self.fig_compare.show()

    if len(self.df1)<=61:
      self.ir_r_2.value = f"{(self.df1['R'].iloc[-1]/self.df1['R'][12]):,.2f}"
      self.pi_r_2.value = f"{0:,.2f}"
      self.r1_r_2.value = f"{self.df1['R'][12]:,.2f}"
      self.r5_r_2.value = f"{self.df1['R'].iloc[-1]:,.2f}"
      self.r10_r_2.value = f"{0:,.2f}"

      self.ir_r_3.value = f"{(self.df2['R'].iloc[-1]/self.df2['R'][12]):,.2f}"
      self.pi_r_3.value = f"{0:,.2f}"
      self.r1_r_3.value = f"{self.df2['R'][12]:,.2f}"
      self.r5_r_3.value = f"{self.df2['R'].iloc[-1]:,.2f}"
      self.r10_r_3.value = f"{0:,.2f}"

      self.ir_s_2.value = f"{(self.df1['S'].iloc[-1]/self.df1['S'][12]):,.2f}"
      self.pi_s_2.value = f"{0:,.2f}"
      self.r1_s_2.value = f"{self.df1['S'][12]:,.2f}"
      self.r5_s_2.value = f"{self.df1['S'].iloc[-1]:,.2f}"
      self.r10_s_2.value = f"{0:,.2f}"

      self.ir_s_3.value = f"{(self.df2['S'].iloc[-1]/self.df2['S'][12]):,.2f}"
      self.pi_s_3.value = f"{0:,.2f}"
      self.r1_s_3.value = f"{self.df2['S'][12]:,.2f}"
      self.r5_s_3.value = f"{self.df2['S'].iloc[-1]:,.2f}"
      self.r10_s_3.value = f"{0:,.2f}"

      self.ir_t_2.value = f"{(self.df1['T'].iloc[-1]/self.df1['T'][12]):,.2f}"
      self.pi_t_2.value = f"{0:,.2f}"
      self.r1_t_2.value = f"{self.df1['T'][12]:,.2f}"
      self.r5_t_2.value = f"{self.df1['T'].iloc[-1]:,.2f}"
      self.r10_t_2.value = f"{0:,.2f}"

      self.ir_t_3.value = f"{(self.df2['T'].iloc[-1]/self.df2['T'][12]):,.2f}"
      self.pi_t_3.value = f"{0:,.2f}"
      self.r1_t_3.value = f"{self.df2['T'][12]:,.2f}"
      self.r5_t_3.value = f"{self.df2['T'].iloc[-1]:,.2f}"
      self.r10_t_3.value = f"{0:,.2f}"

    else:
      self.ir_r_2.value = f"{(self.df1['R'][60]/self.df1['R'][12]):,.2f}"
      self.pi_r_2.value = f"{(self.df1['R'].iloc[-1]/self.df1['R'][12]):,.2f}"
      self.r1_r_2.value = f"{self.df1['R'][12]:,.2f}"
      self.r5_r_2.value = f"{self.df1['R'][60]:,.2f}"
      self.r10_r_2.value = f"{self.df1['R'].iloc[-1]:,.2f}"

      self.ir_r_3.value = f"{(self.df2['R'][60]/self.df2['R'][12]):,.2f}"
      self.pi_r_3.value = f"{(self.df2['R'].iloc[-1]/self.df2['R'][12]):,.2f}"
      self.r1_r_3.value = f"{self.df2['R'][12]:,.2f}"
      self.r5_r_3.value = f"{self.df2['R'][60]:,.2f}"
      self.r10_r_3.value = f"{self.df2['R'].iloc[-1]:,.2f}"

      self.ir_s_2.value = f"{(self.df1['S'][60]/self.df1['S'][12]):,.2f}"
      self.pi_s_2.value = f"{(self.df1['S'].iloc[-1]/self.df1['S'][12]):,.2f}"
      self.r1_s_2.value = f"{self.df1['S'][12]:,.2f}"
      self.r5_s_2.value = f"{self.df1['S'][60]:,.2f}"
      self.r10_s_2.value = f"{self.df1['S'].iloc[-1]:,.2f}"

      self.ir_s_3.value = f"{(self.df2['S'][60]/self.df2['S'][12]):,.2f}"
      self.pi_s_3.value = f"{(self.df2['S'].iloc[-1]/self.df2['S'][12]):,.2f}"
      self.r1_s_3.value = f"{self.df2['S'][12]:,.2f}"
      self.r5_s_3.value = f"{self.df2['S'][60]:,.2f}"
      self.r10_s_3.value = f"{self.df2['S'].iloc[-1]:,.2f}"

      self.ir_t_2.value = f"{(self.df1['T'][60]/self.df1['T'][12]):,.2f}"
      self.pi_t_2.value = f"{(self.df1['T'].iloc[-1]/self.df1['T'][12]):,.2f}"
      self.r1_t_2.value = f"{self.df1['T'][12]:,.2f}"
      self.r5_t_2.value = f"{self.df1['T'][60]:,.2f}"
      self.r10_t_2.value = f"{self.df1['T'].iloc[-1]:,.2f}"

      self.ir_t_3.value = f"{(self.df2['T'][60]/self.df2['T'][12]):,.2f}"
      self.pi_t_3.value = f"{(self.df2['T'].iloc[-1]/self.df2['T'][12]):,.2f}"
      self.r1_t_3.value = f"{self.df2['T'][12]:,.2f}"
      self.r5_t_3.value = f"{self.df2['T'][60]:,.2f}"
      self.r10_t_3.value = f"{self.df2['T'].iloc[-1]:,.2f}"


    self.grid[23:, :8] = self.plot_compare

    self.gui = self.grid
    clear_output(wait=True)
    self.display_gui()

  def clean_data(self, path):
    data = []
    for i in ['D', 'J', 'P']:
      # Case 1: Try reading the column without specifying the sheet name
      try:
        df = pd.read_excel(path, usecols=i, skiprows=9, nrows=122, header=None, names=['data'])
        date = pd.read_excel(path, usecols='B', skiprows=1, nrows=1, header=None)
        self.measure_datetime = pd.to_datetime(date.iloc[0, 0], format='%d%m%Y %I%M%S %p').strftime("%Y-%m-%d %H:%M:%S")
      except ValueError:
      # Case 2: If case 1 fails, try reading from 'Sheet1'
        try:
          df = pd.read_excel(path, usecols=i, skiprows=9, nrows=122, header=None, names=['data'], sheet_name='Sheet1')
          date = pd.read_excel(path, usecols='B', skiprows=1, nrows=1, header=None)
          self.measure_datetime = pd.to_datetime(date.iloc[0, 0], format='%d%m%Y %I%M%S %p', sheet_name='Sheet1').strftime("%Y-%m-%d %H:%M:%S")
        except ValueError:
        # Case 3: If case 2 fails, try reading from 'Data Input'
          try:
            df = pd.read_excel(path, usecols=i, skiprows=9, nrows=122, header=None, names=['data'], sheet_name='Data Input')
            date = pd.read_excel(path, usecols='B', skiprows=1, nrows=1, header=None)
            self.measure_datetime = pd.to_datetime(date.iloc[0, 0], format='%d%m%Y %I%M%S %p', sheet_name='Data Input').strftime("%Y-%m-%d %H:%M:%S")
          except ValueError as e:
            print(f"Error: {e}")

      #case 1: 1st data all column is missing
      if len(df)<61 or (len(df)>61 and len(df)<121):
        df.loc[-1] = [0.01] * len(df.columns)
        df.index = df.index + 1
        df = df.sort_index()

      #case 2: 1st data of some column is missing
      if df['data'].iloc[-1] != df['data'].iloc[-1] and df['data'][0]>=10:
        df = df.drop(df.tail(1).index)
        df.loc[-1] = [0.01] * len(df.columns)
        df.index = df.index + 1
        df = df.sort_index()
      #case 3: some data is missing
      df = df['data'].values
      for i in range(len(df)):
        if df[i] != df[i]:
          df[i] = df[i-1]
      data.append(df)
    df = pd.DataFrame({'R': data[0], 'S': data[1], 'T': data[2]})
    return df

  def check_low_inductance(self, data):
    count_low_indectance = len([x for x in data[12:] if x < 100])
    result = 'Normal' if count_low_indectance == 0 else 'Warning' if (count_low_indectance > 0 and count_low_indectance <=6) else "Danger"
    return result

  def check_full_scale(self, data):
    result = 'Normal' if max(data[:24])<1*10**6 else "Warning"
    return result

  def check_ir_pi(self, data):
    if len(data) == 61:
      ir = data.iloc[-1]/data[12]
      result = 'Normal' if ir >=1.5 else 'Warning' if ir < 1.5 and ir >= 0.9 else 'Danger'
    else:
      pi = data.iloc[-1]/data[12]
      result = 'Normal' if pi >=1.5 else 'Warning' if pi < 1.5 and pi >= 0.9 else 'Danger'

    return result

  def check_mean_inductance(self, data):
    mean = np.mean(data[12:])
    result = 'Normal' if mean >= 1000 else 'Warning' if mean < 1000 and mean >= 250 else "Danger"
    return result

  def check_result(self, data):
    result = self.check_low_inductance(data)
    result = self.check_mean_inductance(data) if result == 'Normal' else result
    result = self.check_ir_pi(data) if result == 'Normal' else result
    result = self.check_full_scale(data) if result == 'Normal' else result
    return result

  def get_result(self, df):
    result_array = []
    for i in ['R', 'S', 'T']:
      result = self.check_result(df[i])
      result_array.append(result)

    if 'Danger' in result_array:
        return 'Danger'
    elif 'Warning' in result_array:
        return 'Warning'
    else:
        return 'Normal'

  def display_gui(self):
    display(self.gui)

# GUI

In [184]:
# @title { vertical-output: true }
output.no_vertical_scroll()
analyzer_gui = Analyzer()
analyzer_gui.display_gui()

GridspecLayout(children=(Dropdown(description='Production Line:', index=3, layout=Layout(grid_area='widget001'…